## The net



![img](./images/net.png)



## Part 1, Forward Propagation



We start off by defining some hyperparameters, constants which establish the
structure and behavior of the network and are not updated as we train.

We're going to look at hours of sleep and hours of study, and predict our test score.

Here X is a 3x2 matrix, and Y is 3 x 1.

$3W_{11} + 5W_{21}$ per each neuron is what we want. We need to figure out
weights for each variable, for each example we have.

We can do this with matrix multiplication (always remember [http://matrixmultiplication.xyz/](http://matrixmultiplication.xyz/)).

$$  \begin{bmatrix}
3 & 5 \\
5 & 1 \\
10 & 2
\end{bmatrix} \begin{bmatrix}
W_{11} & W_{12} & W_{13}\\
W_{21} & W_{22} & W_{23}\\
\end{bmatrix}
$$

$XW^{(1)} = Z^{(2)}$, where $Z^{(2)}$ is the activity of our second layer.

$$
Z^{(2)} = \begin{bmatrix}
3W_{11} + 5W_{21} & 3W_{12} + 5W_{22} & 3W_{13} + 5W_{33} \\
5W_{11} + 1W_{21} & 5W_{12} + 1W_{22} & 5W_{13} + 1W_{33} \\
10W_{11} + 2W_{21} & 10W_{12} + 2W_{22} & 10W_{13} + 2W_{33} \\
\end{bmatrix}
$$

So each entry in Z is a sum of weighted inputs to each neuron. It has size 3x3:
one row for each example, one column for each hidden unit.

Next we will independently apply the activation function to each entry in Z.
We'll use the sigmoid function, leaning on NumPy, which rather conveniently
applies the function element-wise and returns the result with the same
dimensions it was given.

$$
a^{(2)} = \begin{bmatrix}
\sigma(3W_{11} + 5W_{21}) & \sigma(3W_{12} + 5W_{22}) & \sigma(3W_{13} + 5W_{33}) \\
\sigma(5W_{11} + 1W_{21}) & \sigma(5W_{12} + 1W_{22}) & \sigma(5W_{13} + 1W_{33}) \\
\sigma(10W_{11} + 2W_{21}) & \sigma(10W_{12} + 2W_{22}) & \sigma(10W_{13} + 2W_{33}) \\
\end{bmatrix}
$$

So we have $a^{(2)} = f(Z^{(2)})$. We'll then apply one more set of weights to
get our final output, with dimensions 3 x 1, and then run the activation function on that too.

$Z^{(3)} = a^{(2)} W^{(2)}$, and $\hat{y} = f(Z^{(3)})$



In [1]:
import numpy as np
X = np.array([[3, 5],
             [5, 1],
             [10, 2]])
y = np.array([[75], [82], [93]])

# Normalize
X = X/np.amax(X, axis=0)
y = y/100 #Max test score is 100

array([[75],
       [82],
       [93]])

In [1]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

class Neural_Network(object):
    def __init__(self):
        self.inputLayersSize = 2
        self.outputLayersSize = 1
        self.hiddenLayersSize = 3

        self.W1 = np.random.randn(self.inputLayersSize, self.hiddenLayersSize)
        self.W2 = np.random.randn(self.hiddenLayersSize, self.outputLayersSize)

    def forward(self, X):
        self.Z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.Z2)

        self.Z3 = np.dot(self.a2, self.W2)

        y_hat = self.sigmoid(self.Z3)

        return y_hat

    def sigmoid(self, z):
        return 1/(1 + np.exp(-z))

In [1]:
sigmoid(1), sigmoid(np.array([-1, 0, 1])), sigmoid(np.random.randn(3, 3))

NN = Neural_Network()
y_hat = NN.forward(X)
y_hat

array([[0.09039106],
       [0.06463066],
       [0.06301298]])

As we can see, these results are completely terrible compared to our actual test
scores! That is because we are using random weights. Next we need to update our weights.



## Parts 2 - 4: Gradient Descent and Back Propagation



Our goal now is to quantify how wrong our predictions are, figure out how to
update our weights in the right direction, and use our wrongess-quantity to
adjust the weights by some suitable amount.



### Quantifying wrongness with a loss function



We could use several measures of wrongness. For instance, we could just take
$y - \hat{y}$, and that would be fine. Because we're coming up with a single
weight for all our examples, we would average this. Our loss function would thus
be Mean Absolute Error, or MAE.

However, Mean Squared Error is more commonly used, although
[arguments for this are not perfectly convincing](https://stats.stackexchange.com/questions/470626/why-is-using-squared-error-the-standard-when-absolute-error-is-more-relevant-to). It is a little easier to do
calculus on, and most importantly, because it is a convex function, we can be
sure it will be defined at 0, and more extreme errors will get penalized more,
meaning we will learn faster the wronger we are, which is nice.

Mean Squared Error looks like $\sum (y - \hat{y})^2$, and if we divide by 2 to
help us do calculus later it will still have all the properties we want: $J = \sum \frac{1}{2}
(y - \hat{y})^2$.



### Which way to jiggle the weights



Now that we have a loss function, how do we know how to improve our weights?
Random guessing will not work, due to the curse of dimensionality. With 6
weights, assume they might have 1000 possible values each, and we have
$1000^{6}$ guesses to make!

We could also jiggle each weight up or down a little, and see if the whole thing
improves. That will work but be slow.

Fortunately we have a better way! We want to know how our cost function J
changes when $\hat{y}$, or W, changes. This is a derivative! If the derivative
is positive, we are heading in the wrong direction. We'll keep changing until
the derivative of our loss function starts getting worse again.

If we consider one weight at a time, then we want to know how J changes when
just one weight changes, and that's a partial derivative: $\frac{\partial
J}{\partial W}$.

This is one reason we chose our loss function as we did. It's convex, so we will
always know which direction to go in. In higher dimensions, the combination of
all these weights could get us stuck in a local minimum, but if we update our
weights one at a time (stochastic gradient descent), we might be fine anyway.

Anyway, today we will do batch gradient descent, and update them all at once,
based on the partial derivative of each.



### Doing the Math



For $W^{(1)}$ we should get:

\begin{bmatrix}
\frac{\partial J}{\partial W_{11}} & \frac{\partial J}{\partial W_{12}} & \frac{\partial J}{\partial W_{13}}\\
\frac{\partial J}{\partial W_{21}} & \frac{\partial J}{\partial W_{22}} & \frac{\partial J}{\partial W_{33}}\\
\end{bmatrix}

And for $W^{(2)}$ we should get:

\begin{bmatrix}
\frac{\partial J}{\partial W_{11}^{(2)}} \\
\frac{\partial J}{\partial W_{21}^{(2)}} \\
\frac{\partial J}{\partial W_{31}^{(2)}}
\end{bmatrix}

$\frac{\partial J}{\partial W^{(2)}} = \frac{\partial \sum \frac{1}{2} (y -
\hat{y})^2}{\partial W^{(2)}}$, since that's what J is.

The sum here is adding the error from each example to create an overall cost.
The Sum Rule, $\frac{d}{dx}(u + v) = \frac{du}{dx} + \frac{dv}{dx}$, says that
we can move the summation outside our derivative, which is handy.

$\sum \frac{\partial \frac{1}{2} (y - \hat{y})^2}{\partial W^{(2)}}$

We'll come back and add up later.

Well, next we apply the power rule: $\frac{\partial J}{\partial W^{(2)}} = 2 \cdot \frac{1}{2} (y - \hat{y}) = (y - \hat{y})$, and isn't that convenient.

That was the outer function, now to follow the chain rule we must take the
derivative of the inner function.

A better name for back propagation might be, don't stop doing the chain rule, ever!

The $y$ is a constant and goes to 0.

$\frac{\partial J}{\partial W^{(2)}} = (y - \hat{y}) \cdot - \frac{\partial \hat{y}}{\partial W^{(2)}}$

However $\hat{y}$ is itself a function, $\hat{y} = f(z^{(3)})$, and we must apply the chain rule again.

$\frac{\partial \hat{y}}{\partial W^{(2)}} = \frac{\partial \hat{y}}{\partial z^{(3)}} \cdot \frac{\partial z^{(3)}}{\partial W^{(2)}}$

Meaning, again, we want the entire phrase:  
$- (y - \hat{y}) \cdot \frac{\partial \hat{y}}{\partial z^{(3)}} \cdot \frac{\partial z^{(3)}}{\partial W^{(2)}}$

Now, $f(z^{(3)})$ was our sigmoid function, which is $\frac{1}{1 + e^{-z}}$.

To take this derivative, rewrite as $(1 + e^{-z})^{-1}$, and apply the power
rule to get $-(1 + e^{-z})^{-2}$, but then we must apply the product rule to get
$-(1 + e^{-z})^{-2})(\frac{d}{z} 1 + e^{-z})$. Which equals:

$-(1 + e^{-z})^{-2})(\frac{d}{dz}(1 + e^{-z})) = -(1 + e^{-z})^{-2})(-1)(e^{-z}))$

And:
$\frac{e^{-z}}{(1 + e^{-z})^{2}}$. If you do a lot of algebra there, you can
also write this as $\sigma(x) \cdot (1 - \sigma(x))$.

Having found $f'(z^{(3)}$, we can slot that in.

$- (y - \hat{y}) \cdot \frac{e^{-z}}{(1 + e^{-z})^{2}} \cdot \frac{\partial z^{(3)}}{\partial W^{(2)}}$

And we just need our last term. This is how our output - the sum of the
activated (first weights \* inputs) multiplied by the second weights - changes as
the second weights change.

Here's the thing, z3 is a linear function with respect to the W2 weights. Their
relationship is measured by the activations. If
they go up, the values go up. And the derivative of a linear function - it's
just a plain ol' slope like we learned in 5th grade. Here the slope is the
activations, a2.

(this doesn't sound right to me. We change the weights W2, surely)

Our final formula:

$- (y - \hat{y}) \cdot \frac{e^{-z}}{(1 + e^{-z})^{2}} \cdot a^{(2)}$



### Zoom Out a Bit



All that calculus! We wanted to figure out how much our output depended on the
final set of weights we applied. First we had to apply the power rule. Then, we
figured out how much our output was changed by our sigmoid function. Then we
multiplied by the activations.

Remember we'll calculate all this by neuron, by weight. You can think about this
as: we multiply by each activation, because that is how much they each
proportionately contributed to the error. Crucially, this is what lets back
propagation work its magic!

**Our final formula in a nutshell**: *Multiply the size of error, by the derivative of the activation
function, by all our examples with the weights and activation function applied.* 

If you do all that, you will know just how to change each weight.



### Multiply it out



#### The errors



$$  \begin{bmatrix}
y_1 \\
y_2 \\
y_3 \\
\end{bmatrix} - \begin{bmatrix}
\hat{y}_1 \\
\hat{y}_2 \\
\hat{y}_3 \\
\end{bmatrix} = \begin{bmatrix}
y_1 - \hat{y}_1 \\
y_2 - \hat{y}_2 \\
y_3 - \hat{y}_3 \\
\end{bmatrix}
$$

When we applied sigmoid function, we also got a 3x1 matrix, and sigmoidPrime
will have the same shape. In other words $f'(z^{(3)}$ is also 3x1, and we can do
element-wise multiplication.



In [1]:
fakeYs = [[1], [2], [3]]
fakeSigPrime = [[1], [2], [3]]

np.multiply(fakeYs, fakeSigPrime)

#### The size of each error



$$
\begin{bmatrix}
y_1 - \hat{y}_1 \\
y_2 - \hat{y}_2 \\
y_3 - \hat{y}_3 \\
\end{bmatrix} \begin{bmatrix}
f'(z^{(3)}_1) \\
f'(z^{(3)}_2) \\
f'(z^{(3)}_3) \\
\end{bmatrix} = \begin{bmatrix}
\delta^{(3)}_1 \\
\delta^{(3)}_2 \\
\delta^{(3)}_3 \\
\end{bmatrix} = \delta^{(3)}
$$

This is called "the back-propagating error, $\delta^{(3)}$."



#### Multiply together



At this point we want to multiply by $a^{(2)}$, $\delta^{(3)} a^{(2)}$. However,
we've got

$$ a^{(2)} = \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}\\
a_{31} & a_{32} & a_{33}\\
\end{bmatrix}
$$

These matrices don't match. You can't multiply 3x1 with 3x3.

We can make it work by transposing and multiplying, which I'll assume is the
same thing in linear algebra, or something. You can multiply 3x3 with 3x1.

\\[

\begin{bmatrix}
a_{11} & a_{21} & a_{31}\\
a_{12} & a_{22} & a_{32}\\
a_{13} & a_{23} & a_{33}\\
\end{bmatrix} \begin{bmatrix}
\delta^{(3)}_1 \\
\delta^{(3)}_2 \\
\delta^{(3)}_3 \\
\end{bmatrix} = \begin{bmatrix}
a_{11} \: \delta^{(3)}_1 + a_{21} \: \delta^{(3)}_2 + a_{31} \: \delta^{(3)}_3 \\
a_{12} \: \delta^{(3)}_1 + a_{22} \: \delta^{(3)}_2 + a_{32} \: \delta^{(3)}_3 \\
a_{13} \: \delta^{(3)}_1 + a_{23} \: \delta^{(3)}_2 + a_{33} \: \delta^{(3)}_3 \\
\end{bmatrix} 

\\]

And the cool thing here is that the matrix multiplication is adding up across
our examples - there's that summation $\Delta$ we took out earlier!

You can also think of batching gradient as contributing to the overall cost.



### Once More, with Feeling



Now that we have propagated the error backward to our last set of weights, we
need to keep going. How did our first set of weights affect the output? We will
do the same thing, take the partial differential with respect to those weights.

$\frac{\partial J}{\partial W^{(1)}} = \frac{\partial \sum \frac{1}{2} (y -
\hat{y})^2}{\partial W^{(1)}}$, since that's what J is.

Move the summation outside: 
$\sum \frac{\partial \frac{1}{2} (y - \hat{y})^2}{\partial W^{(1)}}$

Then $- (y - \hat{y}) \cdot \frac{\partial \hat{y}}{\partial z^{(3)}} \cdot \frac{\partial z^{(3)}}{\partial W^{(1)}}$

Remember that $- (y - \hat{y}) \cdot \frac{\partial \hat{y}}{\partial z^{(3)}}$
translates to the back-propagating error, $\delta^{(3)}$.

Now we are
calculating the rate of change of z3 with respec to a2, $$\frac{dz}{da_2}$$, so
to speak. The slope here equals the weight value for that synapse.

For some reason he says last time we computed the derivative with respect to the
weights but now we are computing across the synapses.

$\delta^{(3)} \frac{\partial z^{(3)}}{\partial a^{(2)}} \frac{\partial
a^{(2)}}{\partial W^{(1)}}$

The slope is now equal to the weight value for this synapse, which frankly makes
more sense than the a2 being the slope as it was last time.

We'll achieve this by multiplying by the transpose:

$\delta^{(3)} (W^{(2)})^T \frac{\partial a^{(2)}}{\partial W^{(1)}}$


the last term separates into $\frac{\partial a^{(2)}}{\partial z^{(2)}} \frac{\partial z^{(2)}}{\partial W^{(1)}}$

The first part is the whole sigmoid calculation we did before and so we replace
with $f'(z^{(2)})$.

The second part works out as it did last time with $a^{(2)}$, but with X this time.

So we have: 
$x^T \delta^{(3)} (W^{(2)T}) f'(z^{(2)}) = x^T \delta^{(2)}$

I did not fully understand this part and will return once I have reviewed
partial derivatives in the context of the chain rule, as it feels like we're
skipping something. In the meantime I will truck onward with neural nets, as ML
is a practitioner's art.



## Part 4



In [1]:
# Magic adapted from https://mgarod.medium.com/dynamically-add-a-method-to-a-class-in-python-c49204b85bd6
# Makes it more convenient to add as I go in the same notebook
def add_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

@add_method(Neural_Network)
def sigmoidPrime(self, z):
  return np.exp(-z) / ((1 + np.exp(-z))**2)

@add_method(Neural_Network)
def costFunction(self, X, y):
    self.y_hat = self.forward(X)
    J = sum(0.5 * (y - self.y_hat)**2)
    return J

@add_method(Neural_Network)
def costFunctionPrime(self, X, y):
    self.y_hat = self.forward(X)

    self.sigPrimeZ3 = self.sigmoidPrime(self.Z3)
    self.wrongness = y - self.y_hat

    self.delta_3 = np.multiply(-self.wrongness, self.sigPrimeZ3) # element-wise

    dJdW2 = np.dot(self.a2.T, self.delta_3)

    np.dot(X.T, self.delta_3)

    self.sigPrimeZ2 = self.sigmoidPrime(self.Z2)
    self.delta_2 = np.dot(self.delta_3, self.W2.T) * self.sigPrimeZ2

    dJdW1 = np.dot(X.T, self.delta_2)

    return dJdW1, dJdW2

Where does X and y come from? Oh, we're building it a little wonky. I would
first run *forward*, then call costFunctionPrime.

Why `self.sigmoidPrime(self.Z3)`? Oh, the same as any derivative. If y<sub>hat</sub> is a
function of `self.sigmoidPrime(self.Z3)`, then to take the derivative of `y_hat`
with respect to `Z3`, we differentiate the underlying function. Which is what we did.



### Updating



In [1]:
NN.costFunction(X, y)

array([13.64827723])

In [1]:
dJdW1, dJdW2 = NN.costFunctionPrime(X, y)
dJdW1, dJdW2

| array | (((0.13230578 6.74956208 0.26207974) (0.12361459 10.04663462 0.35003568))) | array | (((-0.024160348) (-53.1080763) (-0.0525528428))) |

If we add the gradient to our weights, we go uphill.



In [1]:
W1 = NN.W1
W2 = NN.W2
scalar = 3
NN.W1 = NN.W1 + scalar * dJdW1
NN.W2 = NN.W2 + scalar * dJdW2
cost2 = NN.costFunction(X, y)

NN.W1 = W1
NN.W2 = W2

cost2

array([13.67964497])

If we subtract, we go downhill.



In [1]:
W1 = NN.W1
W2 = NN.W2
scalar = 3
NN.W1 = NN.W1 - scalar * dJdW1
NN.W2 = NN.W2 - scalar * dJdW2
cost3 = NN.costFunction(X, y)

# NN.W1 = W1
# NN.W2 = W2

cost3

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in exp
array([13.63838267])

Why? Imagine the cost function, which is a valley, or a bowl. If the gradient is positive, you are on the right side of the valley. If you add, you go uphill. If the gradient is negative, you are on the left side of the valley. If you add a negative number, you move left, that is uphill. In either case we subtract to go downhill.



## Part 5, Numerical Gradient Checking



So, computers can't do mathematical limits. They can't really do calculus. But
that's okay because calculus is all about getting close enough anyway.

To check our work, we can simply toggle up and down a little and check that our cost function has improved.

We want to add $\epsilon$ to each weight and compute the cost function, then subtract from each weight and compute the cost function.



In [1]:
# this is a helper function to get the weights in a simple format
@add_method(Neural_Network)
def get_params(self):
    params = np.concatenate(( self.W1.ravel(), self.W2.ravel() ))
    return params

# this is a helper function to set the weights
# it works with the getter to roundtrip the weights
# it assumes the weights come in as one giant 1D array
# then we need to chop up that 1d array at the correct places
# np.reshape can run over a 1d array and make rows and cols out of it
@add_method(Neural_Network)
def setParams(self, params):
    W1_start = 0
    W1_end = self.hiddenLayersSize * self.inputLayersSize
    self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayersSize, self.hiddenLayersSize))
    W2_end = W1_end + self.hiddenLayersSize * self.outputLayersSize
    self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayersSize, self.outputLayersSize))

# a helper function to get our gradients, then flatten them into a 1d array
@add_method(Neural_Network)
def computeGradients(self, X, y):
    dJdW1, dJdW2 = self.costFunctionPrime(X, y)
    return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

# This does the hard work of getting the params as a 1d array
# then iterating across them all to add and subtract our epsilon
@add_method(Neural_Network)
def computeNumericalGradient(self, X, y):
    paramsInitial = self.get_params()
    numgrad = np.zeros(paramsInitial.shape)
    perturb = np.zeros(paramsInitial.shape)
    e = 1e-4

    for p in range(len(paramsInitial)):
        #Set perturbation vector: all values in there are now e
        perturb[p] = e
        self.setParams(paramsInitial + perturb)  # matrix addition, element-wise
        loss2 = self.costFunction(X, y)

        self.setParams(paramsInitial - perturb)
        loss1 = self.costFunction(X, y)

        #Compute Numerical Gradient
        numgrad[p] = (loss2 - loss1) / (2*e)

        #Return Params to original value:
        self.setParams(paramsInitial)

    return numgrad

@add_method(Neural_Network)
def checkAccuracy(self):
    grad = self.computeGradients(X,y)
    numgrad = self.computeNumericalGradient(X, y)
    return np.linalg.norm(grad - numgrad) / np.linalg.norm(grad + numgrad)

In [1]:
NN.checkAccuracy()

0.9993355295590062

## Part 6



In [1]:
@add_method(Neural_Network)
def takeScaledStep(self, scalar, X, y):
    dJdW1, dJdW2 = self.costFunctionPrime(X, y)
    self.W1 = self.W1 - scalar * dJdW1
    self.W2 = self.W2 - scalar * dJdW2

@add_method(Neural_Network)
def takeNScaledSteps(self, numSteps, scalar, X, y):
    for step in range(numSteps):
        dJdW1, dJdW2 = self.costFunctionPrime(X, y)
        self.W1 = self.W1 - scalar * dJdW1
        self.W2 = self.W2 - scalar * dJdW2

| array | ((13.59699416)) | array | ((13.59699416)) |

The above was my attempt at training. It does not work at all, unless the inputs are normalized.



In [1]:
# # Normalize
X = X/np.amax(X, axis=0)
y = y/100 #Max test score is 100

At this point, the tutorial implements BFGS. However, I don't understand this portion conceptually, much less the math, and so I will not implement it at this time.

I feel that gradient descent should now work. I don't understand why we need to work about further mathematical optimization.

It will take the second derivative to get the gradient of the gradient, which provides us more information to move downhill more efficiently.

He then uses SciPy's optimize.minimize(), which already implements this function. This is a further strike against this tutorial, as this provides no intuition at all.

However, I am very happy that I am able to train this network. It is not very accurate, even with many steps taken! But it improves several orders of magnitude over our original random weights.

I understand everything in this and am able to train it. Except for the full partial derivatives math involved in taking the gradient. I am optimistic I will soon fully understand this, as my calculus is returning rapidly.



## Part 7



A couple of good notes here. As we have 9 neurons, we would like to have 90 training examples, to reduce overfitting.

We could also implement regularization, where we penalize our model for getting overly complex.



In [1]:
# regularization parameter
Lambda = 0.0001

def costFunction(self, X, y):
    self.y_hat = self.forward(X)
    J = sum(0.5 * (y - self.y_hat)**2) + Lambda/2 * (sum (self.W1 ** 2) + (self.W2 ** 2))
    return J

It is not quite that simple, though. Our model might balance out by weighting one set of weights at the expense of the other.

We need to regularize them. To dJdW2 add Lambda \* self.W2, and to dJdW1 add Lambda \* self.W2.

He also divides by the number of rows in Lambda, the number of examples, I suppose because more examples makes us less likely to overfit and so our cost function should be lower. It is a piece of autoregulation, feedback.

